In [1]:
import pandas as pd
from glob import glob
from sklearn.metrics import roc_auc_score
from analysis.main_analyzer import MainAnalyzer
from analysis.utils import drop_weird_rows
from catboost import CatBoostClassifier, Pool,CatBoost, CatBoostRegressor
from analysis.work_counter_analytics import  WordCounterAnalytics

In [2]:
questions = pd.read_csv("vkcup/data.csv", sep=";", index_col="ID")
test = pd.read_csv("vkcup/test.csv", sep=";", index_col="ID")
test['Answer'] = 0
train = pd.read_csv("vkcup/train.csv", sep=";", index_col="ID")
data = questions.join(train)
drop_weird_rows(data)
data=data[data.index<30000]

In [3]:
main_analyzer = MainAnalyzer()
main_analyzer(data)


/home/ivan/projects/vkcup/analysis/work_counter_analytics.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return np.mean(x2) / np.mean(x1)


In [4]:
train=data[data.index<=30000]
val=data[(data.index>30000)] 

# add_train = val[val.sure_is_bad==1]
# add_train.Answer = 0
# train = pd.concat([train,add_train])


In [5]:
Y = train.Answer
del train['Answer']
X = train

Y_val = val.Answer
del val['Answer']
X_val = val

In [9]:
cat_features=['Question','most_popular_word', 'first_word', 'second_word', 'last_word', 'second_most_popular_word', 'word_after_quote_let']

model = CatBoostClassifier(iterations=4000, 
#                            scale_pos_weight=1/Y.mean(),
                           learning_rate=0.02,
                           task_type="GPU",
                           eval_metric='AUC',
                           cat_features=cat_features,
                           metric_period=1000,
                          )


model.fit(X,
          Y,
#           eval_set=(X_val, Y_val),
          verbose=True)


0:	learn: 0.6910065	total: 52.7ms	remaining: 3m 30s
1000:	learn: 0.8847709	total: 41.9s	remaining: 2m 5s
2000:	learn: 0.9016587	total: 1m 23s	remaining: 1m 23s
3000:	learn: 0.9112232	total: 2m 3s	remaining: 41.1s
3999:	learn: 0.9191740	total: 2m 43s	remaining: 0us


In [11]:
importance = model.get_feature_importance(prettified=True)

In [12]:
importance

,Feature Id,Importances
0,sure_is_bad,36.127808
1,less_popular_word_count,8.293236
2,count_question_sign,6.967240
3,ends_with_question_sign,4.723710
4,end_with_question_condition,4.687601
...,...,...
115,count_quote_type1_right_space,0.000432
116,has_brackets_type4,0.000159
117,has_quote_type4_noname_sign,0.000136
118,count_start_quote_type5,0.000012


In [13]:
val['predict'] = model.predict_proba(val)[:,1]

In [14]:
val[['predict']].to_csv('predicts5.csv',header=False)